In [168]:
filenames = []

import glob

for filepath in glob.iglob('./sample corpus/topics/suicide/*.txt'):
    filenames.append(filepath)
    
    
with open('combined.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [169]:
import re
import string

def sylco(word) :

    word = word.lower()

    # exception_add are words that need extra syllables
    # exception_del are words that need less syllables

    exception_add = ['serious','crucial']
    exception_del = ['fortunately','unfortunately']

    co_one = ['cool','coach','coat','coal','count','coin','coarse','coup','coif','cook','coign','coiffe','coof','court']
    co_two = ['coapt','coed','coinci']

    pre_one = ['preach']

    syls = 0 #added syllable number
    disc = 0 #discarded syllable number

    #1) if letters < 3 : return 1
    if len(word) <= 3 :
        syls = 1
        return syls

    #2) if doesn't end with "ted" or "tes" or "ses" or "ied" or "ies", discard "es" and "ed" at the end.
    # if it has only 1 vowel or 1 set of consecutive vowels, discard. (like "speed", "fled" etc.)

    if word[-2:] == "es" or word[-2:] == "ed" :
        doubleAndtripple_1 = len(re.findall(r'[eaoui][eaoui]',word))
        if doubleAndtripple_1 > 1 or len(re.findall(r'[eaoui][^eaoui]',word)) > 1 :
            if word[-3:] == "ted" or word[-3:] == "tes" or word[-3:] == "ses" or word[-3:] == "ied" or word[-3:] == "ies" :
                pass
            else :
                disc+=1

    #3) discard trailing "e", except where ending is "le"  

    le_except = ['whole','mobile','pole','male','female','hale','pale','tale','sale','aisle','whale','while']

    if word[-1:] == "e" :
        if word[-2:] == "le" and word not in le_except :
            pass

        else :
            disc+=1

    #4) check if consecutive vowels exists, triplets or pairs, count them as one.

    doubleAndtripple = len(re.findall(r'[eaoui][eaoui]',word))
    tripple = len(re.findall(r'[eaoui][eaoui][eaoui]',word))
    disc+=doubleAndtripple + tripple

    #5) count remaining vowels in word.
    numVowels = len(re.findall(r'[eaoui]',word))

    #6) add one if starts with "mc"
    if word[:2] == "mc" :
        syls+=1

    #7) add one if ends with "y" but is not surrouned by vowel
    if word[-1:] == "y" and word[-2] not in "aeoui" :
        syls +=1

    #8) add one if "y" is surrounded by non-vowels and is not in the last word.

    for i,j in enumerate(word) :
        if j == "y" :
            if (i != 0) and (i != len(word)-1) :
                if word[i-1] not in "aeoui" and word[i+1] not in "aeoui" :
                    syls+=1

    #9) if starts with "tri-" or "bi-" and is followed by a vowel, add one.

    if word[:3] == "tri" and word[3] in "aeoui" :
        syls+=1

    if word[:2] == "bi" and word[2] in "aeoui" :
        syls+=1

    #10) if ends with "-ian", should be counted as two syllables, except for "-tian" and "-cian"

    if word[-3:] == "ian" : 
    #and (word[-4:] != "cian" or word[-4:] != "tian") :
        if word[-4:] == "cian" or word[-4:] == "tian" :
            pass
        else :
            syls+=1

    #11) if starts with "co-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:2] == "co" and word[2] in 'eaoui' :

        if word[:4] in co_two or word[:5] in co_two or word[:6] in co_two :
            syls+=1
        elif word[:4] in co_one or word[:5] in co_one or word[:6] in co_one :
            pass
        else :
            syls+=1

    #12) if starts with "pre-" and is followed by a vowel, check if exists in the double syllable dictionary, if not, check if in single dictionary and act accordingly.

    if word[:3] == "pre" and word[3] in 'eaoui' :
        if word[:6] in pre_one :
            pass
        else :
            syls+=1

    #13) check for "-n't" and cross match with dictionary to add syllable.

    negative = ["doesn't", "isn't", "shouldn't", "couldn't","wouldn't"]

    if word[-3:] == "n't" :
        if word in negative :
            syls+=1
        else :
            pass   

    #14) Handling the exceptional words.

    if word in exception_del :
        disc+=1

    if word in exception_add :
        syls+=1     

    # calculate the output
    return numVowels - disc + syls

In [170]:
#utility function for text cleaning
def text_cleaner(text):
  text = re.sub(r'--', ' ', text)
  text = re.sub('[\[].*?[\]]', '', text)
  text = re.sub(r'(\b|\s+\-?|^\-?)(\d+|\d*\.\d+)\b','', text)
  text = ' '.join(text.split())
  return text

In [171]:
with open('./combined.txt') as f:
    with open("./result.txt", "w") as output:
        lines = f.readlines()
        for sentence in lines:
            sentence = sentence[:-1]
            sentence = text_cleaner(sentence)
            words = sentence.split(' ') 

            for i in range(len(words)):
                #words[i] = words[i].lower().strip(".:;?!,")
                words[i].translate(str.maketrans('', '', string.punctuation))

            # then reverse the split string list and join using space 
            reverse_sentence = ' '.join(reversed(words)) + "\n"
            output.write(reverse_sentence)

In [172]:
import nltk


In [173]:
def generateRhymes(input, order):
    pronunDict = dict(nltk.corpus.cmudict.entries()) # CMU Pronunciation Dictionary, contains phonemes used to match rhyming words
    rhymeList = []

    inputSyllables = pronunDict.get(input)

    if inputSyllables:
        rhymeList += [word for word, pronun in pronunDict.items() if pronun[-order:] == inputSyllables[-order:] and word != input]
        return rhymeList


In [201]:
from markovify.chain import Chain, BEGIN, END
import markovify.text
import spacy
nlp = spacy.load("en_core_web_sm")


# Derived class from markovify.Text to generate rhyming sentences from
# a reversed markov chain

DEFAULT_TRIES = 10000
DEFAULT_MAX_OVERLAP_RATIO = 0.49
DEFAULT_MAX_OVERLAP_TOTAL = 15


class rhymeText(markovify.NewlineText):

    '''
    def word_split(self, sentence):
        return ["::".join((word.orth_, word.pos_)) for word in nlp(sentence)]

    def word_join(self, words):
        sentence = " ".join(word.split("::")[0] for word in words)
        return sentence
    '''
    
    def __init__(self, input_text, state_size=1, chain=None, parsed_sentences=None, retain_original=True):
        super().__init__(input_text)
        can_make_sentences = parsed_sentences is not None or input_text is not None
        self.retain_original = retain_original and can_make_sentences
        self.state_size = state_size
        self.rhymeWord = ""
        if self.retain_original:
            self.parsed_sentences = list(self.generate_corpus(input_text))

            # Rejoined text lets us assess the novelty of generated sentences
            self.rejoined_text = self.sentence_join(map(self.word_join, self.parsed_sentences))

            self.chain = chain or Chain(self.parsed_sentences, state_size)

        else:
            if not chain:
                parsed = parsed_sentences or self.generate_corpus(input_text)
            self.chain = chain or Chain(parsed, state_size)

    #4)
    def chooseRhymingWord(self, rhymeWord, matchedWord=None, number=1):
        total = set([])

        rhymeList = generateRhymes(rhymeWord, 2)

        if rhymeList is None or len(rhymeList) == 0 or rhymeList[0] == None:
            return None


        for word in rhymeList:
            if len(total) >= number:
                break
            formatted = (word,)
            formattedPunc = (word + '.',)

            if formatted in self.chain.model:
                matchedWord = formatted
                total.add(matchedWord)

            elif formattedPunc in self.chain.model:
                macthedWord = formattedPunc
                total.add(matchedWord)

        
        if len(total) < number:
            return None
        return list(total)



    def make_short_sentence(self, max_chars, rhymeWord='', min_chars=10, number=1 ):
        tries =  DEFAULT_TRIES



        init_state = self.chooseRhymingWord(rhymeWord, number = number) #4) <===== check this out better
        if init_state == None:
            return None
        
            
        #print("========> ", rhymeWord, init_state)

        values = []
        for x in init_state:
            #print("state", x)
            for _ in range(tries):
                sentence = self.make_sentence(init_state=x)
                #print(sentence)
                #print(min_chars, max_chars, sentence)
                if sentence and sylco(sentence) <= max_chars and sylco(sentence) >= min_chars:
                    values.append(sentence)
                    break
        
        return values

In [209]:
import markovify
import markovify.text

def uppercaseSentence(sentence):
    return sentence[0].upper() + sentence[1:]

def formatReverseSentence(sentence):
    rev = ' '.join(sentence.split()[::-1])
    forwardRhyme = (' ').join(rev.split()[1:])
    return uppercaseSentence(forwardRhyme)

def reverseCorpus(infile, outfile):

    with open(infile, 'r') as f, open(outfile, 'w') as fout:
        for s in f:
            if s is not None:
                words = s.split()
                words.reverse()
                #formattedPunc = words[0][-1] + words[0][:-1]
                #words[0] = formattedPunc
                fout.write("\n" + ' '.join(words))

def buildModels(fFile, rFile):

    with open(fFile) as f:
        forwardText = f.read()

    with open(rFile) as r:
        revText = r.read()

    #print(forwardText)
    forwardModel = markovify.NewlineText(forwardText)
    revModel = rhymeText(revText)

    return (forwardModel, revModel)


            
def buildRhyme(fModel, revModel, min_chars = 10, max_chars=50, size = 1):
    stanza = []

    A_list = []
    while len(A_list) != size:
        A_list.clear()

        while True:
            A = fModel.make_short_sentence(40, min_chars=min_chars) # A
            if A != None:
                break
        
        rhymeWord = A.split()[-1]
        A_list = []
        X = revModel.make_short_sentence(max_chars, rhymeWord, min_chars=min_chars, number=size) #3
        if X != None and len(X) > 0 and X[0] != None:
            A_list = X

    result = [uppercaseSentence(A)]
    for x in A_list:
        result.append(formatReverseSentence(x))

    return result


def buildLetterVerse(fModel, revModel, size, letter=None):
    stanza = []

    A_list = []
    while len(A_list) != size:

        while True:
            A = fModel.make_short_sentence(55) # A
            if A != None and A.startswith(letter):
                break
        
        A_list.append(uppercaseSentence(A))


   

    return A_list


In [177]:
def sonnet(): # ABBA ABBA CDE CDE
    forwardModel, reverseModel = buildModels("./combined.txt", "./result.txt")

    A = buildRhyme(forwardModel, reverseModel, size=4)
    B = buildRhyme(forwardModel, reverseModel, size=4)
    C = buildRhyme(forwardModel, reverseModel, size=2)
    D = buildRhyme(forwardModel, reverseModel, size=2)
    E = buildRhyme(forwardModel, reverseModel, size=2)


    ans =  [
        A[0],
        B[0],
        B[1],
        A[1],
        "\n",
        A[2],
        B[2],
        B[3],
        A[3],
        "\n",
        C[0],
        D[0],
        E[0],
        "\n",
        C[1],
        D[1],
        E[1],

    ]

    return '\n'.join(ans)


    

In [179]:
#ans = sonnet()
#print(ans)

In [180]:
def acrostic(word): # ABBA ABBA CDE CDE
    forwardModel, reverseModel = buildModels("./combined.txt", "./result.txt")

    ans = []
    for s in word:
        ans = ans + buildLetterVerse(forwardModel, reverseModel, size=1 , letter=s)

    return '\n'.join(ans)


    

In [181]:
ans = acrostic('beautiful')
print(ans)

But you dont know how it should be free this way.
Everyday i try to be!
And no one can save my thoughts
Unless the hate me to her attic
That wanted to die..
I have been bare,
For my life away,
Until in the trade
Looking around to see who that is?


In [207]:
def limerick(): # AABBA
    forwardModel, reverseModel = buildModels("./combined.txt", "./result.txt")

    A = buildRhyme(forwardModel, reverseModel, min_chars= 25, max_chars=30, size=4)
    B = buildRhyme(forwardModel, reverseModel, min_chars= 10, max_chars= 15, size=2)

    ans =  [
        A[1],
        A[2],
        B[0],
        B[1],
        A[3]
    ]

    return '\n'.join(ans)



In [210]:
print(limerick())

At life and goodnight.I write my friends try the teeth had to say ppl who loved him up in the the biggest sin
Have sat and cut to fly no need those pills it will i try to smell and although we will begin
I only make them judge
Make you know not crying for all behind.
Ceased, and wat the snow that sees me thy crust and her is apart of the rest has to live in
